In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNet2
from evaluate.cross_evaluator import SemanticCrossEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')

cv_param = {'interval_img_out': 1,
            'num_images': 3,
            'device': th.device("cuda" if th.cuda.is_available() else "cpu"),
            'datasets_path': [ds1_path, ds2_path],
            'results_path': os.path.join(cwd, '../results'),
            'folds': [0, 1, 2, 3],
            'epochs_cv': 100,
            'epochs_ct': 100}

param = {'id': 0,
         'padding_mode': 'reflect',
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': MultiHyperparameter({'type': ['sgd',
                                                    #'adam',
                                                    #'rmsprop',
                                                    #'asgd'
                                                    ],
                                           'lr_factor': [0.5,
                                                         0.75,
                                                         1,
                                                         1.25,
                                                         1.5],
                                           'weight_decay': [0,
                                                            1e-3,
                                                            1e-5]
                                           }).get_full_grid_params(),
         'augment_transform': [{'rotate': False, 'mirror': False, 'translate': False, 'pad': 0},
                               # {'rotate': True, 'mirror': True, 'translate': False, 'pad': 0},
                               # {'rotate': True, 'mirror': True, 'translate': True, 'pad': 16},
                               # {'rotate': True, 'mirror': True, 'translate': True, 'pad': 64}
                               ],
         'num_augments': 100,
         'binarizer_lr': 0.05,
         'batch_size': 1}

# print(MultiHyperparameter({'type': ['sgd', 'adam', 'rmsprop', 'asgd'], 'lr_factor': [0.5, 0.75, 1, 1.25, 1.5], 'weight_decay': [0, 1e-3, 1e-5]}).get_full_grid_params()[0])

unet_hyps = MultiHyperparameter(param)
params = unet_hyps.get_random_params(1)
unet = UNet.__new__(UNet)
#
# models_to_evaluate = [(unet, unet_hyps)]
th.__version__

'1.11.0'

In [ ]:
cte = SemanticCrossEvaluator(unet, cv_param)
#report = cte.cross_test_model(params, cv_param['epochs_ct'], cv_param['epochs_cv'])
cte.train_validate(params[0], ds1_path, [0, 1, 2], [3], 50)

Training:   0%|                                     | 0/50 [00:00<?, ?it/s]
